***Autor: Carlos Emiliano Mendoza Hernández***

Bibliotecas

In [2]:
import numpy as np
import matplotlib.pyplot as plt

Clase generadora de excepciones para los códigos

In [3]:
class InvalidInputMatrix(Exception):
    """
    Exception raised for errors in the input matrix.
    """
    def __init__(self, message):
        super().__init__(message)

#### 1. Solución a la ecuación $\mathbf{Ax=b}$

- Dada la ecuación $\mathbf{Lx=b}$, donde $\mathbf{L}$ es una matriz triangular inferior. Programa una función en Python que encuentre $\mathbf{x}$ dadas $\mathbf{L}$ y $\mathbf{b}$. 

In [4]:
def solve_lx_b(L: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solves the system of linear equations Lx = b, where L is a lower triangular matrix.
    
    Example
    -------
    >>> l = np.array([[2., 0., 0., 0.], 
                      [1., 5., 0., 0.],
                      [4., 5., 2., 0.],
                      [2., 4., 4., 1.]])
    >>> b = np.array([9., 10., 5., 6.]).T
    >>> solve_lx_b(l, b)
    array([[ 4.5 ],
           [ 1.1 ],
           [-9.25],
           [29.6 ]])

    Parameters
    ----------
    L (ndarray) : A lower triangular square matrix
    b (ndarray) : A column vector of the same length as the number of rows in L

    Returns
    ----------
    x (ndarray) : A column vector that solves the system of linear equations Lx = b
    """
    m, n = np.shape(L)
    if m != n:
        raise InvalidInputMatrix("Matrix L is not square")
    n = np.size(b)
    if n != m:
        raise InvalidInputMatrix("Matrix L and vector b have incompatible dimensions")
    x = np.zeros((n,1))
    x[0] = b[0]/L[0,0]
    for i in range (1,n):
        x[i] = (b[i]-L[i,0:i]@x[0:i])/L[i,i]
    return x

- Considera ahora la ecuación $\mathbf{Ux=b}$ donde $\mathbf{U}$ es una matriz triangular superior. Programa en Python una función que encuentre $\mathbf{x}$ dadas $\mathbf{U}$ y $\mathbf{b}$.

In [5]:
def solve_ux_b(U: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solves the system of linear equations Ux = b, where U is an upper triangular matrix.
    
    Example
    -------
    >>> U = np.array([[2., 1., 4., 2.], 
                      [0., 5., 5., 4.],
                      [0., 0., 2., 4.],
                      [0., 0., 0., 1.]])
    >>> b = np.array([9., 10., 5., 6.]).T
    >>> solve_ux_b(u, b)
    array([[14.15],
           [ 6.7 ],
           [-9.5 ],
           [ 6.  ]])

    Parameters
    ----------
    U (ndarray) : An upper triangular square matrix
    b (ndarray) : A column vector of the same length as the number of rows in U

    Returns
    ----------
    x (ndarray) : A column vector that solves the system of linear equations Ux = b
    """
    m, n = np.shape(U)
    if m != n:
        raise InvalidInputMatrix("Matrix U is not square")
    n = np.size(b)
    if n != m:
        raise InvalidInputMatrix("Matrix U and vector b have incompatible dimensions")
    x = np.zeros((n,1))
    x[n-1] = b[n-1]/U[n-1,n-1]
    for i in range (n-2,-1,-1):
        x[i] = (b[i]-U[i,i+1:n]@x[i+1:n,0])/U[i,i]
    return x

- Dada una matriz $\mathbf{A} \in \mathbb{R}^{n \times n}$ invertible, existe la solución a la ecuación $\mathbf{Ax=b}$. Si $\mathbf{A=LU}$, la solución a la ecuación $\mathbf{Ax=b}$ puede encontrarse solucionando primero la ecuación $\mathbf{Ly=b}$ y luego la ecuación $\mathbf{Ux=y}$. Utilizando esto, programa en Python una función que encuentre $\mathbf{x}$ dados $\mathbf{A}$ y $\mathbf{B}$, después de encontrar la factorización $\mathbf{PLU}$ de $\mathbf{A}$.

Funciones auxiliares de PLU

In [6]:
def swap_rows(A: np.ndarray, i: int, j: int) -> np.ndarray:
    B = np.copy(A)
    A[i] = B[j]
    A[j] = B[i]
    return A

In [7]:
def max_index(v: np.array, j: int) -> int:
    v = np.abs(v)
    index = np.argmax(v[j:])
    index += j
    return index

PLU

In [8]:
def plu_fact(A: np.ndarray, epsilon: float=1e-15) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
    """ 
    PLU decomposition of a square matrix A

    Example
    --------
    >>> A = np.array([[0., -1., 4.], [2., 1., 1.], [1., 1., -2.]])
    >>> plu_fact(A)
    (array([[ 0. , 1. , 0.],
            [ 1. , 0. , 0.],
            [ 0. , 0. , 1.]]),
    array([[ 1. ,  0. ,  0. ],
           [ 0. ,  1. ,  0. ],
           [ 0.5, -0.5,  1. ]]),
    array([[ 2. ,  1. ,  1. ],
           [ 0. , -1. ,  4. ],
           [ 0. ,  0. , -0.5]]))

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix

    Returns
    -------
    P (ndarray) : Permutation matrix
    L (ndarray) : Lower triangular matrix
    U (ndarray) : Upper triangular matrix
    """
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('The matrix is not square.')
    P = np.eye(n)
    L = np.zeros((n, n))
    U = np.copy(A)
    for j in range(n-1):
        index = max_index(U[:,j], j)
        if abs(U[index, j]) < epsilon:
            raise InvalidInputMatrix('The matrix is non invertible.')
        P = swap_rows(P, j, index)
        L = swap_rows(L, j, index)
        U = swap_rows(U, j, index)
        for i in range(j+1,n):
            L[i,j] = (U[i,j])/(U[j,j])
            U[i] = U[i] - L[i,j]*U[j]
    L = L + np.eye(n)
    return P, L, U

Los sistemas a resolver son
$$\mathbf{Ly=P^Tb}$$
$$\mathbf{\mathbf{Ux=y}}$$

In [9]:
def solve_sle_plu(A: np.ndarray, b: np.ndarray, epsilon: float=1e-15) -> np.ndarray:
    """
    Solves the system of linear equations Ax = b using PLU factorization

    Example
    --------
    >>> A = np.array([[0., -1., 4.], [2., 1., 1.], [1., 1., -2.]])
    >>> b = np.array([1., 2., 3.])
    >>> solve_sle_plu(A, b)
    array([[ 14.],
           [-21.],
           [ -5.]])

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix
    b (ndarray) : nx1 constant matrix

    Returns
    -------
    x (ndarray) : nx1 solution vector
    """
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix("Matrix A is not square")
    n = np.size(b)
    if n != m:
        raise InvalidInputMatrix("Matrix A and vector b have incompatible dimensions")
    P, L, U = plu_fact(A, epsilon)
    y = solve_lx_b(L, P.T@b)
    x = solve_ux_b(U, y)
    return x

- Realiza lo mismo que en el inciso anterior suponiendo que A es simétrica y definida positiva, es decir, utiliza Cholesky para encontrar $A=LL^T$.

In [21]:
def cholesky_fact(A: np.ndarray, epsilon: float=1e-15) -> np.ndarray:
    """ 
    Cholesky decomposition of a symmetric positive definite matrix A

    Example
    --------
    >>> A = np.array([[4., -1., 1.], [-1., 4.25, 2.75], [1., 2.75, 3.5]])
    >>> cholesky_fact(A)
    array([[ 2. ,  0. ,  0. ],
           [-0.5,  2. ,  0. ],
           [ 0.5,  1.5,  1. ]])

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix

    Returns
    -------
    L (ndarray) : Lower triangular matrix
    """
    if not np.allclose(A, A.T):
        raise InvalidInputMatrix('The matrix is not symmetric.')
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix('The matrix is not square.')
    L = np.zeros((n, n))
    for j in range(n):
        for i in range(j, n):
            if i == j:
                root = A[i, i] - np.sum(L[i,:j]**2)
                if root < 0:
                    raise InvalidInputMatrix('The matrix is non positive definite.')
                root = np.sqrt(root)
                if root < epsilon:
                    raise InvalidInputMatrix('The matrix is non invertible.')
                L[i, i] = root
            elif i > j:
                L[i, j] = (1/L[j, j])*(A[i, j] - np.sum(L[i,:j]*L[j,:j]))
    return L

Los sistemas a resolver son
$$\mathbf{Ly=b}$$
$$\mathbf{L^T=y}$$

In [22]:
def solve_sle_cholesky(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Solves the system of linear equations Ax = b using Cholesky factorization

    Example
    --------
    >>> A = np.array([[4., 12., -16.], [12., 37., -43.], [-16., -43., 98.]])
    >>> b = np.array([1., 2., 3.])
    >>> solve_sle_cholesky(A, b)
    array([[28.58333333],
          [-7.66666667],
          [ 1.33333333]])

    Parameters
    ----------
    A (ndarray) : nxn coefficient matrix
    b (ndarray) : nx1 constant matrix

    Returns
    -------
    x (ndarray) : nx1 solution vector
    """
    m, n = np.shape(A)
    if m != n:
        raise InvalidInputMatrix("Matrix A is not square")
    n = np.size(b)
    if n != m:
        raise InvalidInputMatrix("Matrix A and vector b have incompatible dimensions")
    L = cholesky_fact(A)
    y = solve_lx_b(L, b)
    x = solve_ux_b(L.T, y)
    return x